<a href="https://colab.research.google.com/github/NicoPolazzi/autocomplete/blob/feat%2Fmodel/notebooks/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch

In [ ]:
!git clone https://github.com/NicoPolazzi/autocomplete.git
%cd autocomplete

In [ ]:
#TODO: delete this code cell
!rm -r /content/autocomplete
!git clone -b feat/model --single-branch https://github.com/NicoPolazzi/autocomplete.git
%cd autocomplete

In [ ]:
import sys
sys.path.append('.')

import torch
from torch.utils.data import random_split
from torch.utils.data import DataLoader

from src.dataset import CodeDataset
from src.model import CodeAutocompleteModel
from src.optimization import train_and_evaluate

In [ ]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')

torch.set_default_device(device)
print(f"Using device = {torch.get_default_device()}")

In [ ]:
dataset = CodeDataset(max_length=128, max_samples=1000)

In [ ]:
val_size = int(len(dataset) * 0.2)
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size],
                                          generator=torch.Generator(device='cuda').manual_seed(42))

print(f"obtained a train set with {len(train_dataset)} samples and a test set with \
{len(val_dataset)} samples")

In [ ]:
batch_size = 128
epochs = 10
lr = 1e-3

hidden_dimension = 256
num_layers = 2
vocab_size = len(dataset.vocab)

model = CodeAutocompleteModel(vocab_size, hidden_dimension, num_layers)
train_and_evaluate(model, train_dataset, val_dataset, epochs, lr)